In [2]:
import json
import openai ## Must be updated to the most recent version
import boto3

def fetch_key_from_env(env, keys):
    s3_client = boto3.resource('s3')
    env = '.env.' + env

    config = s3_client.Object('prod-be-config', env).get()['Body'].read().decode("utf-8").split('\n')

    keys_to_return = {}
    for i in config:
        row = i.split('=')
        if len(row) > 1:
            row_key = row[0]
            row_value = row[1]
            if row_key in keys:
                keys_to_return.update({row_key : row_value})
    
    return(keys_to_return)

open_ai_key = fetch_key_from_env('production', 'OPEN_AI_GPT3_ACCESS_KEY')
openai.api_key = open_ai_key['OPEN_AI_GPT3_ACCESS_KEY']


In [57]:

with open('prompt_context.json', 'r') as file:
    prompt_context = json.load(file)

    

recommend_campaign_ideas = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
        {"role": "system", "content": f'''You are a world class marketing planner for a restaurant. You have access to a powerful CRM tool that allows you to create email campaigns to reengage customers in the restaurants customer database. Within these email campaigns, you can include general marketing messages to build awareness, or drive recipients to perform specific actions such as make a reservation, order online, fill out a feedback survey, or redeem a discount offer. Each week, you are given a JSON of data about the business's performance, including sales metrics, review ratings, and number of reviews. In response, you will generate a list of five email campaign prompts that the business could consider running. These prompts should be concise, no more than a sentence or two each.'''},
        {"role": "user", "content": f'''Here are this week's performance metrics: {prompt_context}\n\n Please provide a JSON (using the top level key "campaigns") with an array of five recommended campaigns. Each campaign should come within an object in the main array containing two keys: prompt (containing the campaign prompt), campaign_justification (a brief explanation for why the restaurant should run this campaign based on the provided context). The JSON should be returned with no additional clarifications, commentary or pleasantries.'''}
    ]
    )

campaign_ideas = json.loads(recommend_campaign_ideas['choices'][0]['message']['content'])

campaign_ideas

{'campaigns': [{'prompt': 'Celebrate Singapore National Day at our restaurant and enjoy special discounts!',
   'campaign_justification': "With Singapore National Day approaching, it's a great opportunity to promote our restaurant and offer exclusive discounts to attract customers."},
  {'prompt': 'Order online and get 10% off your meal during rainy days!',
   'campaign_justification': 'With the current rainy weather, encourage customers to order online and offer them a discount to boost online sales.'},
  {'prompt': 'Share your feedback and win a chance to receive a free meal!',
   'campaign_justification': 'Since the number of surveys received has decreased, incentivize customers to provide feedback with a chance to win a free meal, which will help increase engagement.'},
  {'prompt': 'Experience our new outlet in Woodlands and enjoy a complimentary dessert!',
   'campaign_justification': 'With the opening of a new outlet in Woodlands, entice customers to visit by offering them a com

In [69]:
with open('filter_context.json', 'r') as file:
    filter_context = json.load(file)

campaign_recommendation = campaign_ideas['campaigns'][2]

recommend_audience = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
        {"role": "system", "content": f'''You are a data analyst in charge of the customer database for a restaurant. Your job is to create filtered sets of customers from within the customer database that match a simple prompt given to you by the restaurant marketing manager describing an upcoming campaign. Here is a description of the filters that are available to you within the customer database: {filter_context}\n\n When given a prompt for an audience, return a JSON array containing the filters that you would select, and what you would set those filters to in order to create the requested audience in a key titled filters. In a separate key entitled audience_justification include a brief explanation of why you think those filters make sense. When prompted, return only the json, with no extra clarification, pleasantries or greetings.'''},
        {"role": "user", "content": f'''{campaign_recommendation['prompt']}'''}
    ]
    )

campaign_recommendation_plus_audience = [campaign_recommendation, json.loads(recommend_audience['choices'][0]['message']['content'])]
campaign_recommendation_plus_audience

[{'prompt': 'Share your feedback and win a chance to receive a free meal!',
  'campaign_justification': 'Since the number of surveys received has decreased, incentivize customers to provide feedback with a chance to win a free meal, which will help increase engagement.'},
 {'filters': [{'name': 'survey_rating', 'operator': 'equals', 'value': 5},
   {'name': 'issued_voucher_created_at',
    'operator': 'last',
    'value': '30 days'}],
  'audience_justification': 'I would select customers who have rated the restaurant with 5 stars in the survey, indicating their satisfaction. Additionally, I would include customers who have been issued vouchers in the last 30 days to target those who are likely to redeem offers and potentially provide feedback after their free meal.'}]

In [ ]:
cam